In [1]:
import sys
import os
from elasticsearch import Elasticsearch
from zipfile import ZipFile
import re
import pandas as pd
import json
import requests
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import nltk
import pickle
import pytrec_eval
nltk.download('stopwords')
nltk.download('punkt')
from elasticsearch import NotFoundError

es = Elasticsearch()

[nltk_data] Downloading package stopwords to /home/katya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/katya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
input_dir = 'data/docs'
output_dir = 'b075k12'

In [3]:
def strip_punct(s):
    s = re.sub('[^A-Za-z0-9]', ' ', s)
    s = s.lower()
    return " ".join(s.split())

In [4]:
index = 'b075k12_index'

## Create new index, adjust BM25-parameters

In [5]:
request_body = {
    "settings" : {
	        "number_of_shards": 1,
	        "number_of_replicas": 0,
        "similarity": {
      "default": { 
        "type": "BM25",
        "b":0.75,
        "k1":1.2
      }
    }
	    }
	}

es.indices.create(index = index, body = request_body)

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f9db31db350>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f9db31db350>: Failed to establish a new connection: [Errno 111] Connection refused)

In [6]:
with open('bulk_data_json.txt') as f:
    bulk_data = json.load(f)

In [9]:
bulk_data[:100]

[{'index': {'_index': 'b0k5_index',
   '_type': 'doc',
   '_id': 'clueweb12-1913wb-84-09751'}},
 {'query': 'which and has and more and caffeine and coffee and or and tea',
  'title': 'Schabell: Caffee vs <em>Tea</em>, <em>which</em> <em>has</em> <em>more</em> <em>caffeine</em>',
  'title_lem': 'schabell caffee v tea caffeine',
  'num': '31',
  'uuid': 'c90d618b-3dfd-55b9-bbdc-07cdeb747b69',
  'score': 2512.4922,
  'document': 'Caffee vs Tea, which has more caffeine\nThis is an off the wall posting about a topic that has been running around the workplace at about every IT job I have had. Coffee drinkers get headaches in the weekend from too much caffeine and think switching to tea will help. I finally sat down and did a bit of research on this and here are my findings (a hint, the tea drinkers are partially right):\nAt the Indiana University Library you can see the details, but the summary is that "..an average pound of tea has the equivalent caffeine content of two average pounds of co

In [7]:
len(bulk_data)

10060

In [8]:
for x in bulk_data[::2]:
    x['index']['_index']=index
    #print(x)
print(bulk_data[0])

{'index': {'_index': 'b075k12_index', '_type': 'doc', '_id': 'clueweb12-1913wb-84-09751'}}


In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
bulks = chunks(bulk_data, 100)

In [11]:
bulks

<generator object chunks at 0x7fee6699ddd0>

In [12]:
for x in bulks:
    res = es.bulk(index = index, body = x)


#es.search(body={"query": {"match_all": {}}}, index = 'example_index')
#es_indices.get_mapping(index = 'example_index')

/home/katya/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [13]:
res = es.get(index=index,doc_type='doc',id='clueweb12-1214wb-88-29751')

/home/katya/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [14]:
res

{'_index': 'b075k12_index',
 '_type': 'doc',
 '_id': 'clueweb12-1214wb-88-29751',
 '_version': 1,
 '_seq_no': 3633,
 '_primary_term': 1,
 'found': True,
 '_source': {'query': 'what and is and the and difference and between and sex and and and love',
  'title': '<em>Difference</em> <em>Between</em> <em>Love</em> <em>And</em> <em>Sex</em> - Student.com Articles',
  'title_lem': 'difference love sex student article',
  'num': '1',
  'uuid': 'b0054765-4007-52c9-a7c4-70c15891b4cc',
  'score': 2406.7341,
  'document': 'Difference Between Love And Sex\nYou are here:\nStudent.com Articles Student Health & Sexuality Sexual Health Difference Between Love And Sex\nRelated Articles\n<p>Search, Compare, and Apply for student loans at <a href="http://www.simpletuition.com?brnd=studentcom" title="Simpletuition.com" target="_blank">Simpletuition.com</a>.</p>\nPaying For College\nSex VS. Love Love and sex are NOT the same thing. Love is an emotion or a feeling. There is no one definition of love becaus

In [15]:
#es.indices.delete(index='b05k1_index')

In [16]:
es.cat.count(index, params={"format": "json"})

[{'epoch': '1613042176', 'timestamp': '11:16:16', 'count': '3994'}]

## Create different runs

In [17]:
f = open("data/topics_lemmatized.txt", 'rb')
queries_lem = pickle.load(f)
f.close()

In [18]:
def search(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1

    for i in q:
        for x in search_param['query']['bool']['should']:
            if type(i) == dict:
                for key in x['match']:
                    x['match'][key] = i['title']
            else:
                for key in x['match']:
                    x['match'][key] = i

    
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [19]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "title_lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_title', queries_lem, index)

In [20]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_doc', queries_lem, index)

In [21]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "args": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_arg', queries_lem, index)

In [22]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_arg', queries_lem, index)

In [23]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "lem": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc', queries_lem, index)

In [24]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_doc_arg', queries_lem, index)

In [25]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc_arg', queries_lem, index)

## Add synonyms

In [26]:
def search_extended(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1
#['bool']['should']
    for idx, row in q.iterrows():
            
            #for key in x['match']:
                #x['match'][key] = row['query']
        for x in search_param['query']['bool']['should']:
            for key in x["match"]:
                #print(key)
                if "boost" in x['match'][key]:
                    x['match'][key]["query"] = row['query']
                else:
                    x['match'][key] = row['syn']
        #print(search_param)
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [27]:
df_syn = pd.read_csv('data/q_for_syn.tsv', sep = '\t')

In [28]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_syn', df_syn, index)

In [29]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_syn', df_syn, index)

In [30]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_args_syn', df_syn, index)

In [31]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_arg_syn', df_syn, index)

In [32]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_syn', df_syn, index)

In [33]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_arg_syn', df_syn, index)

In [34]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_arg_syn', df_syn, index)

## Evaluate and compare results

In [35]:
def evaluate_run(run_path, qrel_path):
    run = pd.read_csv(run_path, sep = ' ', names = ['topic','Q0','id','rank','score','team'])

    qrel = pd.read_csv(qrel_path, sep = ' ', names = ['topic','Q0','id','relevance'])

    qrels = {}
    for i in range(1,51):
        qrels[str(i)] = {}
    
    for idx, row in qrel.iterrows():
        qrels[str(row['topic'])][row['id']] = row['relevance']
    
    runs = {}
    for i in range(1,51):
        runs[str(i)] = {}
    
    for idx, row in run.iterrows():
        runs[str(row['topic'])][row['id']] = row['score']

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels,{'map_cut', 'ndcg_cut', 'recall', 'P'})

    res = evaluator.evaluate(runs)

    results = {}
    recall = {}
    pres = {}
    for key in res:
        results[key] = res[key]['ndcg_cut_10']
        recall[key] = res[key]['recall_10']
        pres[key] = res[key]['P_10']
        #print(res[key])
    
    filename = re.split('\.', sys.argv[1])
    with open((filename[0]+'_evaluated.json'), 'w') as f:
        json.dump(results, f)
        
    s = sum(results.values())/50
    r = sum(recall.values())/50
    p = sum(pres.values())/50
    if r+p != 0:
        f1 = (2*(r*p)/(r+p))
    else: 
        f1 = 0
    '''
    print("Average ndcg_cut_10: ", s)
    print("Average recall_10: ", r)
    print("Av. precision: ", p)
    print("F1-score: ",f1)
    '''    
    return s, r, p, f1


In [36]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir(output_dir):
    if filename.endswith(".txt"):
        #print(filename)
        s, r, p, f1 = evaluate_run((output_dir+"/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

In [37]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})

### All results

In [38]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.375255,0.233272,0.356,0.281855
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [39]:
df.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.375255,0.233272,0.356,0.281855
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [40]:
df.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.375255,0.233272,0.356,0.281855
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [41]:
df.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.375255,0.233272,0.356,0.281855
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


### Results without doc

In [42]:
df_p = df[~df.run.str.contains("doc")]

In [43]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
8,run_args_syn.txt,0.268146,0.179784,0.260,0.212576
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [44]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
8,run_args_syn.txt,0.268146,0.179784,0.260,0.212576
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [45]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
8,run_args_syn.txt,0.268146,0.179784,0.260,0.212576
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [46]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
1,run_title_arg_syn.txt,0.383752,0.250052,0.372,0.299073
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
8,run_args_syn.txt,0.268146,0.179784,0.260,0.212576
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


### Results for pipeline

In [47]:
df_p = df[~df.run.str.contains("arg")]
df_p = df_p[~df_p.run.str.contains("syn")]

In [48]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268


In [49]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268


In [50]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268


In [51]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268


### Results for synonyms

In [52]:
df_syn = df[~df.run.str.contains("arg")]

In [53]:
df_syn.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [54]:
df_syn.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [55]:
df_syn.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


In [56]:
df_syn.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.434760,0.276890,0.424,0.335007
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
13,run_title_syn.txt,0.353958,0.220843,0.344,0.268995
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
4,run_doc_syn.txt,0.316106,0.192981,0.292,0.232382


### Results for arguments

In [57]:
df_arg = df[~df.run.str.contains("syn")]

In [58]:
df_arg.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
6,run_doc_arg.txt,0.283924,0.189022,0.276,0.224377
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [59]:
df_arg.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
6,run_doc_arg.txt,0.283924,0.189022,0.276,0.224377
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [60]:
df_arg.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
6,run_doc_arg.txt,0.283924,0.189022,0.276,0.224377
9,run_arg.txt,0.266401,0.176880,0.258,0.209874


In [61]:
df_arg.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.440374,0.269583,0.420,0.328386
12,run_title_arg.txt,0.387894,0.251244,0.374,0.300572
0,run_title_doc_arg.txt,0.380404,0.234616,0.356,0.282834
2,run_title.txt,0.341569,0.213226,0.328,0.258444
7,run_doc.txt,0.316231,0.194709,0.294,0.234268
6,run_doc_arg.txt,0.283924,0.189022,0.276,0.224377
9,run_arg.txt,0.266401,0.176880,0.258,0.209874
